In [1]:
from datasets import DatasetDict
from sklearn.model_selection import train_test_split
import csv
from functions_old import *
import datasets
from datasets import load_dataset
import os
import logging
import warnings
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import (
    AutoImageProcessor,
    DinatForImageClassification,
    BertModel,
    AutoTokenizer,
    get_scheduler,
)
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

def collate_fn_test(examples):
    """
    Custom collate function to handle batching of image data and BERT inputs.
    """
    # print(examples)
    pixel_values = torch.stack([example["pixel_values"]
                               for example in examples]).to(device)
    input_ids = torch.stack([example["input_ids"]
                            for example in examples]).to(device)
    attention_mask = torch.stack(
        [example["attention_mask"] for example in examples]).to(device)
    files = [example["file"] for example in examples]

    bert_embeddings = torch.stack(
        [example["bert_embeddings"] for example in examples]).to(device)

    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "files": files,
        "bert_embeddings": bert_embeddings
    }

checkpoint_path = r"D:\Documents\MASC\NLP_EMO\MODELS\best_model_VAL.pt"
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

# Pre-trained model names (ViT + BERT)
image_model_name = "shi-labs/dinat-mini-in1k-224"
bert_model_name = "bert-base-uncased"


dataset_name = 'cairocode/MSPP_TEST_SYL'
# test_dataset = 
test_dataset = load_dataset(dataset_name)['test']

test_dataset.set_transform(test_transforms)

test_loader = DataLoader(
    test_dataset,
    batch_size=50,
    collate_fn=collate_fn_test,
)



image_model = DinatForImageClassification.from_pretrained(
    image_model_name,
    num_labels=1,
    ignore_mismatched_sizes=True,
    problem_type="regression",
).to(device)

image_processor = DinatForImageClassification.from_pretrained(image_model_name).to(device)


tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
base_bert_model = BertModel.from_pretrained(bert_model_name).to(device)

# ----------------------------------------------------------------------
# Initialize the Combined Regression Model
# ----------------------------------------------------------------------
model = CombinedModelsBi(
    image_model=image_model,
    bert_model=bert_model,
    image_feature_dim=512,
    bert_embedding_dim=768,
    combined_dim=512,
    num_labels=1,

).to(device)


if checkpoint_path != None:
    checkpoint = torch.load(checkpoint_path, map_location=device)


model.load_state_dict(torch.load(checkpoint_path))
model.to(device)




# ----------------------------------------------------------------------
# Test Evaluation
# ----------------------------------------------------------------------
print("\nStarting Test Evaluation...")
model.eval()
test_loss = 0.0
all_test_predictions = []
all_test_labels = []
all_filenames = []  # To store filenames

with torch.no_grad():
    test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)
    for batch in test_progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        # labels = batch["labels"].float().to(device)

        # Extract filenames from the batch (assuming "filename" is part of batch)
        filenames = batch["files"]  # Adjust key as necessary
        all_filenames.extend(filenames)

        outputs_dict = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask
        )
        predictions = outputs_dict["logits"].squeeze(-1)

        # loss = criterion(predictions, labels)
        # test_loss += loss.item()

        all_test_predictions.extend(predictions.cpu().numpy())
        # all_test_labels.extend(labels.cpu().numpy())



output_csv_path = os.path.join(output_dir, "valence.csv")
with open(output_csv_path, mode="w", newline="") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Filename", "Valence"])
    for filename, prediction in zip(all_filenames, all_test_predictions):
        writer.writerow([filename, prediction])

print(f"Predictions saved to {output_csv_path}")

# print(dataset_dict)


d:\Documents\MASC\NLP_EMO\.venv312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 26248/26248 [00:01<00:00, 23345.67 examples/s]


ImportError: 
DinatForImageClassification requires the natten library but it was not found in your environment. You can install it by referring to:
shi-labs.com/natten . You can also install it with pip (may take longer to build):
`pip install natten`. Please note that you may need to restart your runtime after installation.
